<a href="https://colab.research.google.com/github/popkaotarbuza/sleep-disorder-diagnosis/blob/dev/ABC-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
github_csv_url = 'https://raw.githubusercontent.com/popkaotarbuza/sleep-disorder-diagnosis/refs/heads/dev/src/data/Sleep_health_and_lifestyle_dataset.csv'
df = pd.read_csv(github_csv_url)

In [3]:
# Подсчёт количества людей с расстройством сна по профессиям
abc = df[df['Sleep Disorder'] != 'None'].groupby('Occupation').size().sort_values(ascending=False)
abc_total = df.groupby('Occupation').size()

abc_df = pd.DataFrame({
    'with_disorder': abc,
    'total': abc_total
}).fillna(0)
abc_df['percent_disorder'] = abc_df['with_disorder'] / abc_df['total'] * 100
abc_df = abc_df.sort_values('with_disorder', ascending=False)

# ABC-классификация
abc_df['cumsum'] = abc_df['with_disorder'].cumsum()
abc_df['cumpercent'] = abc_df['cumsum'] / abc_df['with_disorder'].sum() * 100

def abc_class(x):
    if x <= 80:
        return 'A'
    elif x <= 95:
        return 'B'
    else:
        return 'C'

abc_df['ABC'] = abc_df['cumpercent'].apply(abc_class)
print(abc_df[['with_disorder', 'percent_disorder', 'ABC']])

                      with_disorder  percent_disorder ABC
Occupation                                               
Nurse                            73             100.0   A
Doctor                           71             100.0   A
Engineer                         63             100.0   A
Lawyer                           47             100.0   A
Teacher                          40             100.0   A
Accountant                       37             100.0   B
Salesperson                      32             100.0   C
Scientist                         4             100.0   C
Software Engineer                 4             100.0   C
Sales Representative              2             100.0   C
Manager                           1             100.0   C
